<a href="https://colab.research.google.com/github/Lau-Jr/data-engineering-dump/blob/main/dataframe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

In [ ]:
from pyspark import SparkContext
from pyspark.sql import SparkSession

In [ ]:
sc = SparkContext()
spark = SparkSession(sc)


In [ ]:

data = [
        ('laurent',25,'asha'),
        ('diana',21,'asha'),
        ('yusra',16,'asha'),
        ('france',15,'ngara'),
        ('zion',12,'upendo'),
        ('pu',10,'upendo'),
        ('prince',7,'upendo'),
        ('gato',3,'upendo'),
        ('edo',1,'upendo'),
        ]
names = ['name','age','mother']
data1 = sc.parallelize(data)

In [70]:
watoto = data1.toDF()
new_watoto = spark.createDataFrame(data,schema = names)

In [39]:
house_df = spark.read.csv('/content/sample_data/california_housing_train.csv',header = True,inferSchema =True)
house_df.show(2)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|  -114.31|   34.19|              15.0|     5612.0|        1283.0|    1015.0|     472.0|       1.4936|           66900.0|
|  -114.47|    34.4|              19.0|     7650.0|        1901.0|    1129.0|     463.0|         1.82|           80100.0|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
only showing top 2 rows



In [40]:
ansombe_js = spark.read.json('/content/sample_data/anscombe.json')
ansombe_js.show(2)

+------+----+----+---------------+
|Series|   X|   Y|_corrupt_record|
+------+----+----+---------------+
|  null|null|null|              [|
|     I|10.0|8.04|           null|
+------+----+----+---------------+
only showing top 2 rows



In [45]:
house_df.select(['total_rooms','total_bedrooms']).show(2)

+-----------+--------------+
|total_rooms|total_bedrooms|
+-----------+--------------+
|     5612.0|        1283.0|
|     7650.0|        1901.0|
+-----------+--------------+
only showing top 2 rows



In [46]:
house_df.filter(house_df.total_rooms > 5000).show(3)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|  -114.31|   34.19|              15.0|     5612.0|        1283.0|    1015.0|     472.0|       1.4936|           66900.0|
|  -114.47|    34.4|              19.0|     7650.0|        1901.0|    1129.0|     463.0|         1.82|           80100.0|
|  -116.06|   34.15|              15.0|    10377.0|        2331.0|    4507.0|    1807.0|       2.2466|           66800.0|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
only showing top 3 rows



In [59]:
house_df.groupBy('latitude').count().orderBy('count',ascending = False).count()

840

In [58]:
house_df.groupBy('latitude').count().dropDuplicates().orderBy('count',ascending = False).count()

840

In [71]:
watoto = watoto.withColumnRenamed("_1",'name')

In [73]:
watoto.columns

['name', '_2', '_3']